In [1]:
import gym
import numpy as np
env = gym.make("CartPole-v1")
arr = np.empty((1000, 4))
terminated = truncated = 0
for i in range(1000):

    if i == 0:
        observation, info = env.reset(seed=0)
    elif terminated or truncated:
        observation, info = env.reset()
    else:
        observation, reward, terminated, truncated, info = env.step(env.action_space.sample())
    
    arr[i] = observation

env.close()

In [12]:
np.save('cartpole.npy', arr)

In [2]:
from newmo.wms.fc_vae import FCVAE

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import torch
import pytorch_lightning as pl

In [4]:
class CartPoleDataset(torch.utils.data.Dataset):
    def __init__(self, data: np.ndarray):

        self.X = torch.tensor(data, dtype=torch.float)

    def __len__(self):
        return self.X.shape[0]
    
    def __getitem__(self, index: int):

        return self.X[index]

In [5]:
class CartPoleDataModule(pl.LightningDataModule):

    def __init__(self, data: str, batch_size: int):

        super().__init__()

        self.data = data
        self.batch_size = batch_size

    def setup(self, stage: str):

        self.cartpole_train = CartPoleDataset(self.data)

    def train_dataloader(self):
        return torch.utils.data.DataLoader(self.cartpole_train, batch_size=self.batch_size)

In [13]:
model = FCVAE(x_size=4, latent_size=2)
dm = CartPoleDataModule(arr, 32)

In [14]:
trainer = pl.Trainer(
    max_epochs=100,
    gpus=-1)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [15]:
trainer.fit(model, dm)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type     | Params
-------------------------------------
0 | encoder | _Encoder | 20    
1 | decoder | _Decoder | 12    
-------------------------------------
32        Trainable params
0         Non-trainable params
32        Total params
0.000     Total estimated model params size (MB)


Epoch 99: 100%|██████████| 32/32 [00:00<00:00, 158.74it/s, loss=0.109, v_num=2]
